In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/finaltest.csv
/kaggle/input/file1.csv
/kaggle/input/file2.csv
/kaggle/input/file4.csv
/kaggle/input/file3.csv
/kaggle/input/dataset.csv


In [2]:
import pandas as pd
import numpy as np
import os

import string
from tqdm import tqdm
import re

In [3]:
megaframe=pd.read_csv('/kaggle/input/amazon-ml-images/finaltest.csv')
megaframe

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/amazon-ml-images/finaltest.csv'

In [ ]:
megaframe = megaframe.drop(['Unnamed: 0'], axis=1)
megaframe

In [ ]:
unit_abbreviation_map = {
    'g': 'gram',
    'grams': 'gram',
    'kg': 'kilogram',
    'mg': 'milligram',
    'cup': 'cup',
    'mg': 'milligram',
    'kg': 'kilogram',
    'oz': 'ounce',
    'gallon': 'gallon',
    'v': 'volt',
    'volt': 'volt',
    'w': 'watt',
    'watt': 'watt',
    'lb': 'pound','pound': 'pound',
    'ml': 'millilitre',
    'millilitre': 'millilitre',
    'ft': 'foot',
    'foot': 'foot',
    'ton': 'ton',
    'dl': 'decilitre',
    'decilitre': 'decilitre',
    'in': 'inch',
    'inch': 'inch',
    'l': 'litre',
    'litre': 'litre',
    'mcg': 'microgram',
    'microgram': 'microgram',
    'cm': 'centimetre',
    'centimetre': 'centimetre',
    'qt': 'quart',
    'quart': 'quart',
    'hp': 'horsepower',
    'horsepower': 'horsepower',
    'kw': 'kilowatt',
    'kilowatt': 'kilowatt'
}

In [ ]:
entity_unit_map = {
    'width': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard','m','cm','in','ft'},
    'depth': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard','m','cm','in','ft'},
    'height': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard','m','cm','in','ft'},
    'item_weight': {'gram',
        'kilogram',
        'microgram',
        'milligram',
        'ounce',
        'pound',
        'ton',
        'g',
        'kg',
        'mg','lb'},
    'maximum_weight_recommendation': {'gram',
        'kilogram',
        'microgram',
        'milligram',
        'ounce',
        'pound',
        'ton',
        'kg','g','mg'},
    'voltage': {'kilovolt', 'millivolt', 'volt','v','kv','mv'},
    'wattage': {'kilowatt', 'watt','w','kw'},
    'item_volume': {'centilitre',
        'cubic foot',
        'cubic inch',
        'cup',
        'decilitre',
        'fluid ounce',
        'gallon',
        'imperial gallon',
        'litre',
        'microlitre',
        'millilitre',
        'pint',
        'quart','ml'}
}

allowed_units = {unit for entity in entity_unit_map for unit in entity_unit_map[entity]}
allowed_units

In [ ]:
megaframe['processed_text'] = megaframe['textfromimg'].str.lower()
megaframe.head()

In [ ]:
def convert_floats_to_strings(megaframe):
    def is_float(word):
        try:
            float(word)
            return True
        except ValueError:
            return False

    # Iterate over the 'processed_text' column
    for index, row in megaframe.iterrows():
        text = row['processed_text']
        words = text.split()  # Split the text into individual words

        # Replace floats with strings
        words = [str(float(word)) if is_float(word) else word for word in words]

        # Join the words back into a single string and update the DataFrame
        megaframe.at[index, 'processed_text'] = ' '.join(words)

    return megaframe


In [ ]:
megaframe['processed_text'] = megaframe['processed_text'].astype(str)
megaframe['processed_text'] = megaframe['processed_text'].str.replace('o', '0').str.replace('z', '2').str.replace('"','in')

In [ ]:
def extract_numeric_units(text, allowed_units):
    # Regular expression to match patterns like 2.4g, 5.8g, etc.
    pattern = re.compile(r'(\d+(?:\.\d+)?)\s*([a-zA-Z]+|")(?:\/(\d+(?:\.\d+)?)\s*([a-zA-Z]+|"))?')
    
    # Find all matches
    matches = pattern.findall(text)
    
    # Filter matches based on the allowed units for this specific entity
    extracted = []
    for match in matches:
        number, unit, number2, unit2 = match
        
        # Check if the first unit is in the allowed units for the entity
        if unit in allowed_units:
            extracted.append(f"{number} {unit}")
        
        # If a second number/unit pair exists, check it as well
        if number2 and unit2 in allowed_units:
            extracted.append(f"{number2} {unit2}")
    
    return extracted

# Initialize the new column in your DataFrame
megaframe['extracted_values'] = [[] for _ in range(len(megaframe))]

# Iterate through each row and extract values based on the entity_name
for index, row in megaframe.iterrows():
    text = row['processed_text']
    entity_name = row['entity_name']
    
    # Get the allowed units for the current entity_name from entity_unit_map
    allowed_units_for_entity = entity_unit_map.get(entity_name, set())  # Default to an empty set if no mapping exists
    
    # Extract numeric-unit pairs based on the allowed units for the specific entity
    extracted = extract_numeric_units(text, allowed_units_for_entity)
    
    # Append extracted values to the new column
    megaframe.at[index, 'extracted_values'] = extracted

In [ ]:
megaframe[['entity_name','extracted_values']].head(20)

In [ ]:
def count_empty_extracted_values(megaframe):
    empty_count = 0
    for values in megaframe['extracted_values']:
        if not values:  # Check if the list is empty
            empty_count += 1
    return empty_count

# Usage
empty_lists_count = count_empty_extracted_values(megaframe)
print(f"Number of empty lists in 'extracted_values': {empty_lists_count}")

In [ ]:
def filter_non_empty_extracted_values(megaframe):
    # Filter rows where 'extracted_values' is not empty
    filtered_megaframe = megaframe[megaframe['extracted_values'].apply(bool)]
    return filtered_megaframe

non_empty_rows = filter_non_empty_extracted_values(megaframe)
non_empty_rows

In [ ]:
# Function to map abbreviated units to full names
def map_units(value, entity_name):
    # Extract the number and unit from the value (e.g., '500g', '0.729 g')
    match = re.match(r"([0-9.]+)\s*([a-zA-Z]+)", value)
    if match:
        number, unit = match.groups()
        # Convert the number to float to maintain precision
        number_float = float(number)
        # Map unit using the abbreviation map
        full_unit = unit_abbreviation_map.get(unit, unit)  # Default to the same unit if not found in map
        return f"{number_float} {full_unit}"
    return value

# Function to find the most frequent value and map its unit to full name
# Function to compare the first and last value in the list and return the greater one
def get_predicted_value(row):
    entity_name = row['entity_name']
    values = row['extracted_values']
    
    # Check if extracted_values is None or empty
    if values is None or len(values) == 0:
        return None
    
    # Convert all values to their full unit names
    full_values = [map_units(value, entity_name) for value in values]
    
    # Extract the numeric part of the first and last value
    first_value = float(re.match(r"([0-9.]+)", full_values[0]).group(1))
    last_value = float(re.match(r"([0-9.]+)", full_values[-1]).group(1))
    
    # Compare and return the greater value (with its full unit)
    if first_value >= last_value:
        return full_values[0]
    else:
        return full_values[-1]


In [ ]:
megaframe['prediction'] = megaframe.apply(get_predicted_value, axis=1)

print(megaframe[['entity_name','prediction']].head(20))

In [ ]:
output=megaframe[['index','prediction']]
output.to_csv('test_out_new.csv', index=False)